In [1]:
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
# from torchvision.transforms import ToTensorfrom 
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
class PointCloudDataset(Dataset):
    def __init__(self, pts_file, split=0):
        points = np.loadtxt(pts_file, delimiter=' ')
        points = np.delete(points, -2, 1)
        points = np.delete(points, -2, 1)
        points = np.delete(points, -2, 1)

        if split == 1:
            points = points[:len(points)//2]
        elif split == 2:
            points = points[len(points)//2:]

        print(points.shape)
        # do i need to min max intensity, return number, number of returns, etc.? probably not
        for i in range(3):
            dim_min, dim_max = min(points[:,i]), max(points[:,i])
            points[:,i] = (points[:,i] - dim_min) / (dim_max - dim_min)
        self.data = points
        
    def __len__(self):
        return len(self.data) // 25000
        
    def __getitem__(self, idx):
        # Return batches of 2500 points
        xyzirn = self.data[idx * 25000: (idx + 1) * 25000, :-1]  # x, y, z, ***intensity, return number, number of returns***
        label = self.data[idx * 25000: (idx + 1) * 25000, -1] == 8

        xyzirn = torch.from_numpy(xyzirn.T).float()
        label = torch.tensor(label).long()
        
        return xyzirn, label

training_data = PointCloudDataset(r"C:\Users\and13375\Documents\3D Point Segmentation\Test\Vaihingen\3DLabeling\Vaihingen3D_Traininig.pts")
validation_data = PointCloudDataset(r"C:\Users\and13375\Documents\3D Point Segmentation\Test\Vaihingen\3DLabeling\Vaihingen3D_EVAL_WITH_REF.pts", split=1)
testing_data = PointCloudDataset(r"C:\Users\and13375\Documents\3D Point Segmentation\Test\Vaihingen\3DLabeling\Vaihingen3D_EVAL_WITH_REF.pts", split=2)
train_dataloader = DataLoader(training_data, batch_size=10, shuffle=True) # can/should i use shuffle, try lowering it?
validation_dataloader = DataLoader(validation_data, batch_size=10, shuffle=False)
test_dataloader = DataLoader(testing_data, batch_size=10, shuffle=False)

num_classes = 2

(753876, 4)
(205861, 4)
(205861, 4)


In [3]:
print(next(iter(train_dataloader))[0].size())

torch.Size([10, 3, 25000])


In [4]:
for i, data in enumerate(train_dataloader):
    print(data[1].size())

torch.Size([10, 25000])
torch.Size([10, 25000])
torch.Size([10, 25000])


In [5]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([10, 3, 25000])
Labels batch shape: torch.Size([10, 25000])


In [6]:
import open3d as o3d
import numpy as np
from collections import Counter

def plot_batch(entire_view = True):
    # Assuming pc is your point cloud data, in shape (N, 3)
    pc_num = 0

    if entire_view:
        pc = training_data.data[:, :3]
        labels = training_data.data[:, 3] == 8
    else:
        pc = training_data[pc_num][0].T.view(-1,3).numpy()
        labels = training_data[pc_num][1].numpy()

    print(Counter(labels), len(labels))

    # Define colors for each label
    color_map = {0: [0.5, 0.5, 0.5],  # Gray color for label 0
                1: [1.0, 0.0, 0.0]}  # Red color for label 1

    # Map each label to a color
    colors = np.array([color_map[label] for label in labels])

    # Create point cloud
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(pc)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([point_cloud])
# plot_batch(entire_view = False)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device = 'cpu'
print(f"Using {device} device")

Using cpu device


In [8]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F


class STN3d(nn.Module):
    def __init__(self):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,self.k*self.k).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True, feature_transform = True):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        print('AFTER MAXPOOL:', x.size())
        x = x.view(-1, 1024)
        print('AFTER RESHAPE:', x.size())
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat


class PointNetDenseCls(nn.Module):
    def __init__(self, k = 2, feature_transform=True):
        super(PointNetDenseCls, self).__init__()
        self.k = k
        self.feature_transform=feature_transform
        self.feat = PointNetfeat(global_feat=False, feature_transform=feature_transform)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        print(x.size())
        n_pts = x.size()[2]
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans, trans_feat
    
# model = PointNetDenseCls(k=2).to(device)

In [9]:
# for name, module in model.named_modules():
#     if name == 'mlp8.1':
#         print(name)
#     # print(name)

In [10]:
def feature_transform_regularizer(trans):
    d = trans.size(1)
    I = torch.eye(d).unsqueeze(0).to(device)
    loss = torch.linalg.norm(I - torch.bmm(trans, trans.transpose(2,1)), dim=(1,2))
    loss = torch.mean(loss)
    return loss

In [11]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.cross_entropy_loss = nn.CrossEntropyLoss()

    def forward(self, inputs, targets, trans):
        d = trans.size(1)
        I = torch.eye(d).unsqueeze(0).to(device)
        loss = torch.linalg.norm(I - torch.bmm(trans, trans.transpose(2,1)), dim=(1,2))
        loss = torch.mean(loss)
        return self.cross_entropy_loss(inputs, targets) + loss

## Online Code

In [12]:
from sklearn.metrics import f1_score
from collections import Counter


writer = SummaryWriter()
loss_func = CustomLoss()
classifier = PointNetDenseCls().to(device)

optimizer = optim.Adam(classifier.parameters(), lr=0.0005, betas=(0.9, 0.999))
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

num_batch = len(training_data)

for epoch in range(80):
    # scheduler.step()
    classifier.train()
    train_loss, train_f1, train_acc = 0.0, 0.0, 0.0
    predictions, labels = np.array([]), np.array([])
    for i, data in enumerate(train_dataloader, 1):
        points, target = data
        points, target = points.to(device), target.to(device)
        optimizer.zero_grad()
        pred, _, trans_feat = classifier(points)
        pred = pred.view(-1, num_classes)
        target = target.view(-1, 1).squeeze()
        # print(pred.size(), target.size())

        loss = F.nll_loss(pred, target)
        loss += feature_transform_regularizer(trans_feat) * 0.001
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        predictions = np.append(predictions, pred.max(1)[1].cpu())
        labels = np.append(labels, target.cpu())

    train_f1 = f1_score(predictions, labels)
    train_acc = sum(predictions == labels)/float(len(labels))
    train_loss /= len(train_dataloader)

    classifier.eval()
    valid_loss, valid_f1, valid_acc = 0.0, 0.0, 0.0
    predictions, labels = np.array([]), np.array([])
    for i, data in enumerate(validation_dataloader):
        points, target = data
        points, target = points.to(device), target.to(device)
        pred, _, trans_feat = classifier(points)
        pred = pred.view(-1, num_classes)
        target = target.view(-1, 1).squeeze()

        loss = F.nll_loss(pred, target)
        loss += feature_transform_regularizer(trans_feat) * 0.001
        valid_loss += loss.item()

        predictions = np.append(predictions, pred.max(1)[1].cpu())
        labels = np.append(labels, target.cpu())

    valid_f1 = f1_score(predictions, labels)
    valid_acc = sum(predictions == labels)/float(len(labels))
    valid_loss /= len(validation_dataloader)

    writer.add_scalars('losses', {'training':train_loss, 'validation':valid_loss}, global_step=epoch)
    writer.add_scalars('f1 scores', {'training':train_f1, 'validation':valid_f1}, global_step=epoch)

    print(f'[{epoch}] train loss: {train_loss} accuracy: {train_acc} f1 score: {train_f1}')
    print(f'[{epoch}] validation loss: {valid_loss} accuracy: {valid_acc} f1 score: {valid_f1}')
    print()

writer.flush()
writer.close()

torch.Size([10, 3, 25000])
torch.Size([10, 3, 25000])
torch.Size([10, 3, 25000])
torch.Size([8, 3, 25000])
[0] train loss: 0.878406802813212 accuracy: 0.46159466666666665 f1 score: 0.2947615870676135
[0] validation loss: 0.6651950478553772 accuracy: 0.86571 f1 score: 0.0

torch.Size([10, 3, 25000])
torch.Size([10, 3, 25000])
torch.Size([10, 3, 25000])
torch.Size([8, 3, 25000])
[1] train loss: 0.8077305952707926 accuracy: 0.5614546666666667 f1 score: 0.3399338145724337
[1] validation loss: 0.6592779755592346 accuracy: 0.86571 f1 score: 0.0

torch.Size([10, 3, 25000])
torch.Size([10, 3, 25000])
torch.Size([10, 3, 25000])
torch.Size([8, 3, 25000])
[2] train loss: 0.7422040700912476 accuracy: 0.694816 f1 score: 0.3695934780213727
[2] validation loss: 0.6463310718536377 accuracy: 0.86571 f1 score: 0.0

torch.Size([10, 3, 25000])
torch.Size([10, 3, 25000])
torch.Size([10, 3, 25000])
torch.Size([8, 3, 25000])
[3] train loss: 0.7042587598164877 accuracy: 0.7586546666666667 f1 score: 0.27666707

In [17]:
from sklearn.metrics import f1_score

outputs = None
points = None
x = y = None
def test_PointNet(pointnet, test_dataloader, device):
    global outputs, points, x, y
    pointnet.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for idx, (points, labels) in enumerate(test_dataloader):
            points, labels = points.to(device), labels.to(device)
            outputs, _, _ = pointnet(points)
            _, predicted = torch.max(outputs.data, 2)
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()
            x = labels.view(-1).to('cpu').numpy()
            y = predicted.view(-1).to('cpu').numpy()
            f1 = f1_score(x, y)
            print('F1 score: ', f1)
    
test_PointNet(classifier, train_dataloader, device)

torch.Size([10, 3, 25000])
F1 score:  0.6544759713176224
torch.Size([10, 3, 25000])
F1 score:  0.558924848825347
torch.Size([10, 3, 25000])
F1 score:  0.2633047210300429


In [14]:
print(sum(x))
print(sum(y))
print(outputs)

45569
16694
tensor([[[-1.1129e-01, -2.2508e+00],
         [-1.1381e-01, -2.2296e+00],
         [-1.2146e-01, -2.1682e+00],
         ...,
         [-8.0524e-03, -4.8258e+00],
         [-2.2432e-01, -1.6047e+00],
         [-1.5096e+00, -2.4975e-01]],

        [[-4.2806e-02, -3.1724e+00],
         [-1.8053e-02, -4.0234e+00],
         [-1.3528e-02, -4.3098e+00],
         ...,
         [-1.7949e-02, -4.0292e+00],
         [-1.8144e-02, -4.0185e+00],
         [-1.8349e-02, -4.0073e+00]],

        [[-3.1332e-01, -1.3131e+00],
         [-3.0351e-01, -1.3403e+00],
         [-2.9631e-01, -1.3608e+00],
         ...,
         [-3.9249e-03, -5.5424e+00],
         [-4.0114e-03, -5.5206e+00],
         [-4.0681e-03, -5.5066e+00]],

        ...,

        [[-4.7488e-01, -9.7275e-01],
         [-4.7584e-01, -9.7118e-01],
         [-4.7705e-01, -9.6920e-01],
         ...,
         [-1.3204e-02, -4.3338e+00],
         [-1.3236e-02, -4.3314e+00],
         [-1.3118e-02, -4.3403e+00]],

        [[-1.4316e-02,

In [15]:
print(f"Labels has {sum(x == 0)} zeroes and {sum(x == 1)} ones")
print(f"Predictions has {sum(y == 0)} zeroes and {sum(y == 1)} ones")


# writer.add_graph(model, points)
# writer.flush()
# writer.close()

Labels has 204431 zeroes and 45569 ones
Predictions has 233306 zeroes and 16694 ones


73.4029268292683% on roofs (5)
86.77414634146342% on trees (8)